<a href="https://colab.research.google.com/github/slrlab-tech/cantonese-tone-passage/blob/main/MFA_Cantonese.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%%writefile install_mfa.sh
#!/bin/bash

## a script to install Montreal Forced Aligner (MFA)

root_dir=${1:-/tmp/mfa}
mkdir -p $root_dir
cd $root_dir

# download miniconda3
wget -q --show-progress https://repo.anaconda.com/miniconda/Miniconda3-latest-Linux-x86_64.sh
bash Miniconda3-latest-Linux-x86_64.sh -b -p $root_dir/miniconda3 -f

# create py38 env
$root_dir/miniconda3/bin/conda create -n aligner -c conda-forge openblas python=3.8 openfst pynini ngram baumwelch -y
source $root_dir/miniconda3/bin/activate aligner

# install mfa, download kaldi
pip install montreal-forced-aligner # install requirements
pip install git+https://github.com/MontrealCorpusTools/Montreal-Forced-Aligner.git # install latest updates

mfa thirdparty download

echo -e "\n======== DONE =========="
echo -e "\nTo activate MFA, run: source $root_dir/miniconda3/bin/activate aligner"
echo -e "\nTo delete MFA, run: rm -rf $root_dir"
echo -e "\nSee: https://montreal-forced-aligner.readthedocs.io/en/latest/aligning.html to know how to use MFA"

Writing install_mfa.sh


In [ ]:
# download and install mfa
INSTALL_DIR="/tmp/mfa" # path to install directory

!bash ./install_mfa.sh {INSTALL_DIR}
!source {INSTALL_DIR}/miniconda3/bin/activate aligner; mfa align --help

Miniconda3-latest-L 100%[===================>]  63.62M   243MB/s    in 0.3s    
PREFIX=/tmp/mfa/miniconda3
Unpacking payload ...
Solving environment: - \ | done

## Package Plan ##

  environment location: /tmp/mfa/miniconda3

  added / updated specs:
    - _libgcc_mutex==0.1=main
    - _openmp_mutex==4.5=1_gnu
    - brotlipy==0.7.0=py39h27cfd23_1003
    - ca-certificates==2021.7.5=h06a4308_1
    - certifi==2021.5.30=py39h06a4308_0
    - cffi==1.14.6=py39h400218f_0
    - chardet==4.0.0=py39h06a4308_1003
    - conda-package-handling==1.7.3=py39h27cfd23_1
    - conda==4.10.3=py39h06a4308_0
    - cryptography==3.4.7=py39hd23ed53_0
    - idna==2.10=pyhd3eb1b0_0
    - ld_impl_linux-64==2.35.1=h7274673_9
    - libffi==3.3=he6710b0_2
    - libgcc-ng==9.3.0=h5101ec6_17
    - libgomp==9.3.0=h5101ec6_17
    - libstdcxx-ng==9.3.0=hd4cf53a_17
    - ncurses==6.2=he6710b0_1
    - openssl==1.1.1k=h27cfd23_0
    - pip==21.1.3=py39h06a4308_0
    - pycosat==0.6.3=py39h27cfd23_0
    - pycparser==2.

In [ ]:
!source {INSTALL_DIR}/miniconda3/bin/activate aligner; \
mfa align input cantonese_pronunciation.dict cantonese_model.zip output -j4 -c

All required kaldi binaries were found!
Cleaning old directory!
/root/Documents/MFA/input/align.log
INFO - Setting up corpus information...
INFO - Number of speakers in corpus: 1, average number of utterances per speaker: 1.0
INFO - Parsing dictionary without pronunciation probabilities without silence probabilities
INFO - Creating dictionary information...
INFO - Setting up training data...
Generating base features (mfcc)...
Calculating CMVN...
INFO - Done with setup!
INFO - Performing first-pass alignment...
INFO - Calculating fMLLR for speaker adaptation...
INFO - Performing second-pass alignment...
INFO - All done!


In [ ]:
# # download and unpack ljs dataset
# !echo "download and unpack ljs dataset"
# !mkdir -p ./ljs; cd ./ljs; wget -q --show-progress https://data.keithito.com/data/speech/LJSpeech-1.1.tar.bz2
# !cd ./ljs; tar xjf LJSpeech-1.1.tar.bz2

In [ ]:
# # install sox tool
# !sudo apt install -q -y sox
# # convert to 16k audio clips
# !mkdir ./wav
# !echo "normalize audio clips to sample rate of 16k"
# !find ./ljs -name "*.wav" -type f -execdir sox --norm=-3 {} -r 16k -c 1 `pwd`/wav/{} \;
# !echo "Number of clips" $(ls ./wav/ | wc -l)

In [ ]:
# # create transcript files from metadata.csv
# lines = open('./ljs/LJSpeech-1.1/metadata.csv', 'r').readlines()
# from tqdm.auto import tqdm
# for line in tqdm(lines):
#   fn, _, transcript = line.strip().split('|')
#   ident = fn
#   open(f'./wav/{ident}.txt', 'w').write(transcript)

# # this is an example transcript for LJ001-0001.wav
# !cat ./wav/LJ001-0001.txt

In [ ]:
# # download a pretrained english acoustic model, and english lexicon
# !wget -q --show-progress https://github.com/MontrealCorpusTools/mfa-models/raw/main/acoustic/english.zip
# !wget -q --show-progress http://www.openslr.org/resources/11/librispeech-lexicon.txt

In [ ]:
# # FINALLY, align phonemes and speech
# !source {INSTALL_DIR}/miniconda3/bin/activate aligner; \
# mfa align -t ./temp -c -j 4 ./wav librispeech-lexicon.txt ./english.zip ./ljs_aligned
# # output files are at ./ljs_aligned
# !echo "See output files at ./ljs_aligned"